# QAOA
It is almost the same as the VQE algorithm, but we use QAOA-specific ansatz for combinatorial optimization problem.

## What we learn
1. How QAOA works
2. Implement QAOA with a simple example

## Quantum Adiabatic Computation
QAOA uses a mechanism called quantum adiabatic computation of time evolution.

Let t be the current time and T be the overall schedule. This time we use two Hamiltonian. Let $H_{start}$ be the initial Hamiltonian to prepare, and $H_{final}$ be the Hamiltonian containing the problem you want to solve. At that time, we gradually switch from the initial Hamiltonian to the Hamiltonian of the problem we want to solve, as shown below.

$$
H_{temp} = (1-\frac{t}{T})H_{start} + \frac{t}{T}H_{final}
$$

If $T\rightarrow\infty$, the ground state will be the following eigenstates

$$
H_{temp}\mid \psi \rangle = E_{0temp}\mid \psi \rangle
$$

The process of replacing the two Hamiltonians is created by using the time evolution operator.

$$
\mid \psi_{t+1} \rangle = e^{-iHt}  \mid \psi_t \rangle
$$

## Quantum Circuit
The quantum circuit consists of

1. Initial state preparation
2. QAOA Ansatz

```
|0> ---[Initial state preparation]---[QAOA Ansatz]---[Measurement]
```

The far left is the initial state preparation, the next is the first step of QAOA Ansatz, and the next is the second step of QAOA Ansatz. Finally, take a measurement and find the solution.

## Two Hamiltonians.
This time, we will use an initial Hamiltonian (mixer Hamiltonian) and a Hamiltonian (cost Hamiltonian) for the problem we want to solve. The mixer Hamiltonian is chosen differently depending on the constraints of the problem. The cost Hamiltonian formulates the value you want to minimize in the problem.

## Preparing the mixer Hamiltonian and the initial state
The mixer Hamiltonian is meant to determine how to explore the problem. In quantum annealing, X is used as a mixer Hamiltonian to search for the answer while inverting the value of the qubit.

If the mixer Hamiltonian is determined by the constraints of the problem, the initial state is determined. The initial state preparation sets the eigen state of the mixer Hamiltonian. If you want to use an X-mixer with the above X as a mixer Hamiltonian, choose the default eigenstate $\mid \psi \rangle$ originally set to a state such as $\mid + \rangle$.

## Cost Hamiltonian
The cost Hamiltonian changes the formulation depending on the problem. Based on a social problem, a type of problem called a combinatorial optimization problem is dropped onto a model called the Ising model of physics. The cost Hamiltonian uses the Z operator's expected values of +1 and -1 to express all the conditions in question using the Z or ZZ operators.

```
hamiltonian = -Z(0) - Z(0)*Z(1)
```

Set the "bias" before a single Z, and set the "weight" before multiple Zs.

## QAOA Ansatz and the Hamiltonian's time evolution
In QAOA Ansatz, the cost Hamiltonian for setting up the problem and the mixer Hamiltonian for determining the search method are stored in a form called the time evolution operator.

The time evolution of the $X$ gate makes use of gates that contain angles, such as the $RX(\theta)$ gate. The two-qubit will also include the time evolution of the $XX$ gate, which includes an angle, such as $RXX(\theta)$.

## Number of steps.
Quantum adiabatic calculations are usually analog processes discretized using time evolution operators. QAOA can also increase the accuracy by setting the number of steps. The number of steps corresponds to the number of iterations in the QAOA Ansatz.

## X-mixer
A typical mixer is the X-mixer.

$$
X\mid 0 \rangle = 
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
\begin{bmatrix}
1\\
0
\end{bmatrix}
=
\begin{bmatrix}
0\\
1
\end{bmatrix}
$$

This is used to invert the value of a qubit by acting on a single qubit. The initial state is also an H-gate on a single qubit to create a $\mid + \rangle$ state.

## XY Mixer
The XY mixer switches between the two states. (XX+YY)/2 is

$$
X_0X_1 + Y_0Y_1 = 
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
\otimes
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
+
\begin{bmatrix}
0&-i\\
i&0
\end{bmatrix}
\otimes
\begin{bmatrix}
0&-i\\
i&0
\end{bmatrix}
=
\begin{bmatrix}
0&0&0&1\\
0&0&1&0\\
0&1&0&0\\
1&0&0&0
\end{bmatrix}
+
\begin{bmatrix}
0&0&0&-1\\
0&0&1&0\\
0&1&0&0\\
-1&0&0&0
\end{bmatrix}
$$

so,

$$
(X_0X_1 + Y_0Y_1)/2 
=
\begin{bmatrix}
0&0&0&0\\
0&0&1&0\\
0&1&0&0\\
0&0&0&0
\end{bmatrix}
$$

Here, you will explore the answer exchanging 01 and 10 states. Initially, the entangled states of 01 and 10 are selected as eigenstates. Also,

$$
(X_0X_1 - Y_0Y_1)/2 = 
\begin{bmatrix}
0&0&0&1\\
0&0&0&0\\
0&0&0&0\\
1&0&0&0
\end{bmatrix}
$$

This is exchange 00 and 11 states. The entangled state of 00 and 11 is selected for inisital state.

## Example: Quantum Annealing
QaoaAnsatz is initially set to quantum annealing, and I've used X in the mixer, which is initially set the eigenstate to |+>. The initial state|+> is achieved by applying an Hadamar gate H to all the qubits.

```
|0> --H--  --RZZ--RZ--RX- -RZZ--RZ--RX--  --[Measurement]
|0> --H--  --*--------RX- -*--------RX--  --[Measurement]
```

In [27]:
import numpy as np
from blueqat import Circuit
from blueqat.utils import qaoa
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q

#mixer
Xmixer = X[0]+X[1]

#initial state
Xinit = Circuit(2).h[0,1]

h = -Z(0) -Z(0)*Z(1)
step = 2

result = qaoa(h, step, Xinit, Xmixer)
result.circuit.run(shots=10)

Counter({'00': 9, '01': 1})

If the initial state and the mixer are not specified, quantum annealing is employed.

In [28]:
result = qaoa(h, step)
result.circuit.run(shots=10)

Counter({'00': 10})

## Example: XY Mixer
Next, we'll try using an XY mixer for the same problem. Specify the entanglement state as the initial state. The mixer is given as a Hamiltonian of (XX+YY)/2.

In [29]:
#mixer
XYmixer = 0.5*X[0]*X[1] + 0.5*Y[0]*Y[1]

#initial state
XYinit = Circuit().h[0].cx[0,1].x[0]

result = qaoa(h, step, XYinit, XYmixer)
b = result.circuit.run(shots=10)

print(b)

Counter({'01': 10})


We would have liked to see (0,0), but we put a constraint on 01 and 10, so XY mixer is trying to find the one that is close to the correct answer among the answers of 01 and 10.
